In [70]:
import json
import os
import pandas as pd
import numpy as np
import requests as req
import random

## Reporte de ventas

In [16]:
# Crear un df con la columnas "fechas - pasado - final"
reporte = pd.DataFrame(columns=['fecha', 'Monto Pasado', 'Monto Actual'])
reporte.to_csv('reporte.csv', index=False)

## Configuración

In [71]:
# Datos 

dic_datos = {
    "fechas" : ["2024-10-04"],
    "montos_reporte": [11962],
    "monto_limite": 6000,
    "servidor" : "127.0.0.1",
    "productos" : ["034003", "042035"]
}

# Crear un directorio para guardar los datos

## API

In [72]:
api = f'http://{dic_datos["servidor"]}:8000/florcatorce/'

## Obtener las ventas consumiento la API

In [73]:
i = 0
for fecha in dic_datos["fechas"]:
    ventas = req.post(f'{api}ventas', json={"fecha": fecha})
    # Crear un marco de trabajo
    df = pd.DataFrame(ventas.json())
    # Cambiar los tipos de datos
    df["efectivo"] = df["efectivo"].astype(float)
    df["tarjeta"] = df["tarjeta"].astype(float)
    df["total"] = df["total"].astype(float)
    # Obtener los folios para hacer mantenimiento
    df_folios = df[(df["efectivo"] >= 120) & (df["tarjeta"] == 0) & (df["facturado"] == False)]
    # Crear un array de folios
    folios_array = np.array(df_folios["folio"])
    random.shuffle(folios_array)
    total_folios = np.size(folios_array)
    # Crear un array de efectivo
    efectivo_array = np.array(df_folios["efectivo"])
    total_efectivo = np.sum(efectivo_array)
    # Abrir el archivo de reporte
    df_reporte = pd.read_csv('reporte.csv')
    # Empezar con el mantenimiento de ventas
    efectivo_folios = 0
    for folio in folios_array:
        producto_select = np.random.choice(dic_datos["productos"])
        try:
            update = req.patch(f'{api}mantenimiento/{folio}', json={"cantidad_articulos": 1,"producto": producto_select})
        except Exception as e:
            print(e)
            exit()
        efectivo_folio = df_folios[df_folios["folio"] == folio]["efectivo"].values[0]
        efectivo_folios += efectivo_folio
        
        if producto_select == "034003":
            efectivo_folios -= 80
        elif producto_select == "042035":
            efectivo_folios -= 40
        
        # Determinar si el proceso sigue o no
        flag = dic_datos["montos_reporte"][i] - efectivo_folios
        if flag > 6000 and flag < 7050:
            print(f'El mantenimiento acabo con {flag} en la fecha {fecha}')
            break
    i += 1

El mantenimiento acabo con 6889.799999999999 en la fecha 2024-10-02
El mantenimiento acabo con 6964.0 en la fecha 2024-10-05
